In [165]:
def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input() -> list[str]:
    return get_lines_from_file('./test_input')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func

In [166]:
from typing import Tuple, Generator, NamedTuple, Iterable
from itertools import chain

Grid = NamedTuple('Grid', [('data', list[int]), ('width', int), ('height', int)])
Tree = NamedTuple('Tree', [('x', int), ('y', int)])

def column(grid: Grid, x: int) -> Generator[int, None, None]:
    return [cell(grid, x, y) for y in range(grid.height)]

def row(grid: Grid, y: int) -> Generator[int, None, None]:
    return [cell(grid, x, y) for x in range(grid.width)]

def cell(grid: Grid, x: int, y: int) -> Tuple[int, int]:
    return grid.data[y * grid.width + x]

def get_visible_idxs(tree_line: Iterable[int]) -> list[int]:
    highest_yet = -1
    visible_idxs = []

    for idx, height in enumerate(tree_line):
        if height > highest_yet:
            visible_idxs.append(idx)
            highest_yet = height

    return visible_idxs


def solution1(input: list[str]) -> int:
    grid_data = list(map(lambda c: int(c), chain(*input)))
    grid = Grid(grid_data, len(input[0]), len(input))

    visible_trees = set()

    for y in range(grid.height):
        tree_row = row(grid, y)
        visible_trees.update(map(lambda i: Tree(i, y), get_visible_idxs(tree_row)))
        visible_trees.update(map(lambda i: Tree(grid.width - 1 - i, y), get_visible_idxs(tree_row[::-1])))

    for x in range(grid.width):
        tree_col = column(grid, x)
        visible_trees.update(map(lambda i: Tree(x, i), get_visible_idxs(tree_col)))
        visible_trees.update(map(lambda i: Tree(x, grid.height - 1 - i), get_visible_idxs(tree_col[::-1])))

    #print(sorted(list(visible_trees), key=lambda t: (t.x, t.y)))
    return len(visible_trees)


In [167]:
def visible_trees(tree_line: list[int], ref_height: int) -> int:
    tree_count = 0
    for height in tree_line:
        if height >= ref_height:
            tree_count += 1
            break
        else:
            tree_count += 1
    return tree_count
    
def scenic_score(tree_line: list[int], ref_idx: int) -> int:
    scenic_score = 1
    ref_height = tree_line[ref_idx]
    
    if ref_idx > 0:
        scenic_score *= visible_trees(tree_line[ref_idx-1::-1], ref_height)
    if ref_idx < len(tree_line) - 1:
        scenic_score *= visible_trees(tree_line[ref_idx+1:], ref_height)
    
    return scenic_score

def solution2(input: list[str]) -> int:
    grid_data = list(map(lambda c: int(c), chain(*input)))
    grid = Grid(grid_data, len(input[0]), len(input))

    highest_score = 0

    for x in range(grid.width):
        for y in range(grid.height):
            score = scenic_score(row(grid, y), x) * scenic_score(column(grid, x), y)
            if score > highest_score:
                highest_score = score
    
    return highest_score

In [168]:
solutions = [
    solution1,
    solution2,
]

test_results = [
    get_int_from_file('./test_result1'),
    get_int_from_file('./test_result2'),
]

def run_test(idx) -> bool:
    res = solutions[idx-1](get_test_input())
    test_res = test_results[idx-1]
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx):
    sol = solutions[idx-1](get_input())
    print(f'The solution for part {idx} is: {sol}')

if run_test(1):
    run_solution(1)
    print('\nOn to part 2...\n')
    if run_test(2):
        run_solution(2)

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 1782

On to part 2...

Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 2 is: 474606
